In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

from astropy.io import fits
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

In [ ]:
fwhm_star = 0.04 * 2.355 # arcsec

In [ ]:
def fwhm_int_simple(fwhm_obs, fwhm_star):
    return (fwhm_obs**2 + fwhm_star**2) ** 0.5

In [ ]:
CAT_PATH = '/home/alberto/cosmos/ista/COLA1/catalogs/F200W_cat/F200W_det_O3_doublets.fits'
cat = fits.open(CAT_PATH)[1].data
cat.columns

In [ ]:
FWHM_int = fwhm_int_simple(cat['FWHM_WORLD'] * 3600, fwhm_star) * u.arcsec

R_eff = FWHM_int / 2 * cosmo.kpc_proper_per_arcmin(cat['z_O3doublet_combined_n_1'])
R_eff = R_eff.to(u.kpc).value

In [ ]:
z_Arr = cat['z_O3doublet_combined_n_1']

fig, ax = plt.subplots()

mask_COLA1 = cat['NUMBER_1_1'] == 9269

ax.errorbar(z_Arr, R_eff,
            fmt='o')
ax.errorbar(z_Arr[mask_COLA1], R_eff[mask_COLA1],
           marker='*', ms=20, c='r')
ax.errorbar([6.6], fwhm_star / 2 * cosmo.kpc_proper_per_arcmin(6.6).to(u.kpc / u.arcsec).value,
            fmt='o')

ax.set_ylabel(r'$R_{\rm eff}$ [pkpc]')
ax.set_xlabel('Redshift')

plt.show()

In [ ]:
R_eff_percs = np.nanpercentile(R_eff, [16, 50, 84])
print(R_eff_percs[1], -R_eff_percs[1] + R_eff_percs)

In [ ]:
def app_m_to_flux_nu(m):
    """
    :param m: apparent magnitude
    :return: the (observed) flux density
    """
    return 10 ** (-(m + 48.60) / 2.5)


def flux_to_lum(flux, z):
    """
    :param flux: flux
    :param z: redshift
    :return: luminosity
    """
    return flux * 4 * np.pi * cosmo.luminosity_distance(z).to("cm").value ** 2

def flux_nu_to_lum_nu(flux_nu, z):
    """
    :param flux_nu: the (observed) flux density
    :param z: redshift
    :return: the (intrinsic) luminosity density
    """
    return flux_to_lum(flux_nu, z) / (1 + z)

def continuum_powerlaw(wavelength, norm, beta, redshift):
    return norm*(wavelength/(1500.*(1+redshift)))**beta

In [ ]:
from jpasLAEs.utils import flux_to_mag
import sys
sys.path.insert(0, '..')
from dust_correction import SFR_XION_dust_corrected

# Compute SFR
# For now we assume the same value of xi_ion as in Matthee2023.
SFR_C_UV_COLA1 = 43.51
# SFR_C_Hb_COLA1 = 41.78
SFR_C_Hb_COLA1 = 41.78


SFR_C_UV = 43.46
SFR_C_Hb = 41.64

SFR = np.zeros(len(cat))
SFR_err_up = np.zeros(len(cat))
SFR_err_down = np.zeros(len(cat))

EBV = np.zeros(len(cat))
EBV_err_up = np.zeros(len(cat))
EBV_err_down = np.zeros(len(cat))

XION = np.zeros(len(cat))
XION_err_up = np.zeros(len(cat))
XION_err_down = np.zeros(len(cat))

SFR_UV = np.zeros(len(cat))
SFR_UV_err_up = np.zeros(len(cat))
SFR_UV_err_down = np.zeros(len(cat))

MUV = np.zeros(len(cat))

for src in range(len(cat)):
    if cat['NUMBER_1_1'][src] == 9269:
        this_SFR_C_Hb = SFR_C_Hb_COLA1
        this_SFR_C_UV = SFR_C_UV_COLA1
        this_Hb = 4.10216
        this_Hb_err = 0.16479
        this_Hg = 2.11797
        # this_Hg =  this_Hb * 0.4
        this_Hg_err = 0.20597
        this_beta = -3.18
        this_beta_err = 0.18
        this_norm = 5.149
        this_norm_err = 0.3715
    else:
        this_SFR_C_Hb = SFR_C_Hb
        this_SFR_C_UV = SFR_C_UV
        this_Hb = cat['f_Hb_1'][src]
        this_Hb_err = cat['f_Hb_err_1'][src]
        this_Hg = 0.4 * this_Hb
        this_Hg_err = 0
        this_beta = cat['beta'][src]
        this_beta_err = cat['beta_err'][src]
        this_norm = cat['norm'][src]
        this_norm_err = cat['norm_err'][src]


    this_redshift = cat['z_O3doublet_combined_n_1'][src]
    N_iter = 5000
    this_rand_beta = np.random.normal(this_beta, this_beta_err, size=N_iter)
    this_rand_norm = np.random.normal(this_norm, this_norm_err, size=N_iter)

    this_obs_1500 = 1500 * (1 + this_redshift)

    args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
    this_f_UV = continuum_powerlaw(*args) * 1e-20
    
    this_distmod = cosmo.distmod(this_redshift).value

    this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
    this_m_UV[~np.isfinite(this_m_UV)] = -99

    this_M_UV_Arr = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

    this_M_UV = np.nanmedian(this_M_UV_Arr)
    this_M_UV_err = (np.nanpercentile(this_M_UV_Arr, 84) - np.nanpercentile(this_M_UV_Arr, 16)) * 0.5

    MUV[src] = this_M_UV

    SFR_Hb_percs, SFR_UV_percs, XION_percs = SFR_XION_dust_corrected(this_Hg, this_Hg_err,
                                                                     this_Hb, this_Hb_err,
                                                                     this_M_UV, this_M_UV_err,
                                                                     this_redshift, this_SFR_C_Hb,
                                                                     this_SFR_C_UV,
                                                                     N_iter=100000)

    this_EBV = SFR_XION_dust_corrected(this_Hg, this_Hg_err,
                                       this_Hb, this_Hb_err,
                                       this_M_UV, this_M_UV_err,
                                       this_redshift, this_SFR_C_Hb,
                                       this_SFR_C_UV,
                                       N_iter=100000, return_EBV=True)

    EBV_percs = np.nanpercentile(this_EBV, [16, 50, 84])

    EBV[src] = EBV_percs[1]
    EBV_err_up[src] = EBV_percs[2] - EBV_percs[1]
    EBV_err_down[src] = EBV_percs[1] - EBV_percs[0]

    SFR[src] = SFR_Hb_percs[1]
    SFR_err_up[src] = SFR_Hb_percs[2] - SFR_Hb_percs[1]
    SFR_err_down[src] = SFR_Hb_percs[1] - SFR_Hb_percs[0]

    SFR_UV[src] = SFR_UV_percs[1]
    SFR_UV_err_up[src] = SFR_UV_percs[2] - SFR_UV_percs[1]
    SFR_UV_err_down[src] = SFR_UV_percs[1] - SFR_UV_percs[0]

    XION[src] = XION_percs[1]
    XION_err_up[src] = XION_percs[2] - XION_percs[1]
    XION_err_down[src] = XION_percs[1] - XION_percs[0]

print('COLA1')
print(EBV[mask_COLA1][0], EBV_err_up[mask_COLA1][0], EBV_err_down[mask_COLA1][0])
print(SFR[mask_COLA1][0], SFR_err_up[mask_COLA1][0], SFR_err_down[mask_COLA1][0])
print(SFR_UV[mask_COLA1][0], SFR_UV_err_up[mask_COLA1][0], SFR_UV_err_down[mask_COLA1][0])
print(XION[mask_COLA1][0], XION_err_up[mask_COLA1][0], XION_err_down[mask_COLA1][0])
print()
print('C1F')
mask_Hb_sn = cat['f_Hb_1'] / cat['f_Hb_err_1'] > 3
print(np.median(EBV[mask_Hb_sn]))
print('SFR Hb', np.median(SFR[mask_Hb_sn]), -np.median(SFR[mask_Hb_sn]) + np.percentile(SFR[mask_Hb_sn], [16, 84]))
print('SFR UV', np.median(SFR_UV[mask_Hb_sn]), -np.median(SFR_UV[mask_Hb_sn]) + np.percentile(SFR_UV[mask_Hb_sn], [16, 84]))
print(np.median(XION[mask_Hb_sn]))
print(np.median(MUV[mask_Hb_sn]))

In [ ]:
Sigma_SFR = np.log10(SFR / (2 * np.pi * R_eff**2))
Sigma_SFR_err_up = np.log10((SFR + SFR_err_up) / SFR)
Sigma_SFR_err_down = np.log10(SFR / (SFR - SFR_err_down))

Sigma_SFR_UV = np.log10(SFR_UV / (2 * np.pi * R_eff**2))
Sigma_SFR_UV_err_up = np.log10((SFR_UV + SFR_UV_err_up) / SFR_UV)
Sigma_SFR_UV_err_down = np.log10(SFR_UV / (SFR_UV - SFR_UV_err_down))

print('Sigma(Hbeta)', Sigma_SFR[mask_COLA1], Sigma_SFR_err_up[mask_COLA1], Sigma_SFR_err_down[mask_COLA1])
print('Sigma(UV)', Sigma_SFR_UV[mask_COLA1], Sigma_SFR_UV_err_up[mask_COLA1], Sigma_SFR_UV_err_down[mask_COLA1])

In [ ]:
# Ion3 https://ui.adsabs.harvard.edu/abs/2018MNRAS.476L..15V/abstract
N_iter = 10000

r_eff_ion3 = np.random.uniform(1, 1.3, size=N_iter)
MUV_ion3 = np.random.normal(-22.2, 0.15, size=N_iter)
fnu_ion3 = 10**(-0.4 * (MUV_ion3 + 48.6))  # erg/s/Hz/cm2 at 10 pc
LUV_ion3 = fnu_ion3 * (4. * np.pi * (10. * 3.08568 * 10**18)**2)  # erg/s/Hz
EBV_ion3 = np.random.uniform(0, 0.1, size=N_iter)
dustcorr_UV = 10**(0.4*(EBV_ion3)*8.68)
SFRUV = LUV_ion3 * dustcorr_UV / 10**43.46 * 299792458 / 1500e-10
print(f'{np.nanpercentile(SFRUV, [16, 50, 84])}')
Sigma_UV_ion3 = SFRUV / (2 * np.pi * r_eff_ion3**2)
print(f'{np.log10(np.nanpercentile(Sigma_UV_ion3, [16, 50, 84]))}')


# Ion2 https://iopscience.iop.org/article/10.3847/0004-637X/825/1/41
# https://ui.adsabs.harvard.edu/abs/2016A%26A...585A..51D/exportcitation
N_iter = 10000

r_eff_ion2 = np.random.normal(0.34, 0.025, size=N_iter)
fnu_ion2 = app_m_to_flux_nu(24.6) * 1e18
LUV_ion2 = fnu_ion2 * (4. * np.pi * (10. * 3.08568 * 10**18)**2)  # erg/s/Hz
EBV_ion2 = 0.
dustcorr_UV = 10**(0.4*(EBV_ion2)*8.68)
SFR_C_UV_ion2 = np.random.uniform(43.46, 43.46, size=N_iter)
SFRUV = LUV_ion2 * dustcorr_UV / 10**SFR_C_UV_ion2 * 299792458 / 1500e-10
print(f'{np.nanpercentile(SFRUV, [16, 50, 84])}')
Sigma_UV_ion2 = SFRUV / (2 * np.pi * r_eff_ion2**2)
print(f'{np.log10(np.nanpercentile(Sigma_UV_ion2, [16, 50, 84]))}')

# MUSE HUDF Leakers (Kerutt+23, Kerutt+22)
k22 = fits.open('Kerutt22.fits')[1].data
k22_id_list = [1181371, 3052076, 109004028, 122021111, 126049137,
               1521589, 3452147, 4062373, 4172404, 5622786, 119004004, 122032127]
k22_EBV_list = [0.11, 0.2, 0.15, 0.02, 0.24, 0.04, 0.18, 0.21, 0.14, 0.3, 0.19, 0.13]
k22_EBV_err_list = [0.02, 0.0, 0.03, 0.01, 0.06,
                    0.02, 0.04, 0.02, 0.05, 0.02, 0.03, 0.05]
Sigma_k22 = []
Sigma_k22_err_up = []
Sigma_k22_err_down = []
redshift_k22 = []
for iii, this_id in enumerate(k22_id_list):
    src = np.where(k22['ID'] == this_id)[0][0]
    this_k22_MUV = np.random.normal(
        k22['MUV'][src], k22['e_MUV'][src], size=N_iter)
    this_r_eff = np.random.normal(
        k22['Re'][src], k22['e_Re'][src], size=N_iter)
    redshift_k22.append(k22['z'][src])
    this_fnu_k22 = 10**(-0.4 * (this_k22_MUV + 48.6))  # erg/s/Hz/cm2 at 10 pc
    this_LUV_k22 = this_fnu_k22 * \
        (4. * np.pi * (10. * 3.08568 * 10**18)**2)  # erg/s/Hz
    this_EBV = np.random.normal(k22_EBV_list[iii], k22_EBV_err_list[iii], size=N_iter)
    this_dustcorr_UV = 10**(0.4*(this_EBV)*8.68)
    this_SFRUV = this_LUV_k22 * this_dustcorr_UV / 10**43.46 * 299792458 / 1500e-10
    this_Sigma_UV_k22 = this_SFRUV / (2 * np.pi * this_r_eff**2)
    Sigma_k22.append(np.nanmedian(this_Sigma_UV_k22))
    Sigma_k22_err_up.append(np.nanpercentile(
        this_Sigma_UV_k22, 84) - Sigma_k22[-1])
    Sigma_k22_err_down.append(-np.nanpercentile(this_Sigma_UV_k22,
                              16) + Sigma_k22[-1])

# Izotov+16: https://ui.adsabs.harvard.edu/abs/2016MNRAS.461.3683I/abstract
f_Hb_list = np.array([30.8, 12.2, 38.9, 27.0]) * 1e-16
Dl_list = (np.array([1888, 1736, 1582, 1975]) * u.Mpc).to(u.cm).value
r_eff_list = np.array([0.59, 1.44, 0.86, 1.34])
I16_z_list = [0.3419, 0.3181, 0.3937, 0.3557]
SFR = f_Hb_list * 4 * np.pi * Dl_list**2 / 10**41.78 * 2.86
Sigma_i16 = np.log10(SFR / (2 * np.pi * r_eff_list**2))

# Vanzella+22, Sunburst arc
V22_MUV = -20.3
V22_r = 3**0.5 # kpc
fnu_V22 = 10**(-0.4 * (V22_MUV + 48.6))  # erg/s/Hz/cm2 at 10 pc
LUV_V22 = fnu_V22 * (4. * np.pi * (10. * 3.08568 * 10**18)**2)  # erg/s/Hz
dustcorr_UV_V22 = 10**(0.4*(0.15)*8.68)
SFRUV_V22 = LUV_V22 * dustcorr_UV_V22 / 10**43.46 * 299792458 / 1500e-10
Sigma_V22 = np.log10(SFRUV_V22 / (2 * np.pi * V22_r**2))
print(Sigma_V22)

In [ ]:
mask_Hb_sn = cat['f_Hb_1'] / cat['f_Hb_err_1'] > 3

fig, ax = plt.subplots(figsize=(6, 5))

ax.errorbar(z_Arr[mask_COLA1], Sigma_SFR[mask_COLA1],
            yerr=[Sigma_SFR_err_up[mask_COLA1], Sigma_SFR_err_down[mask_COLA1]],
            fmt='D', ls='', ms=10, mfc='crimson', mec='k', zorder=901,
            mew=1.5, label=r'COLA1 (H$\beta$)')
ax.errorbar(z_Arr[mask_COLA1], Sigma_SFR_UV[mask_COLA1],
            yerr=[Sigma_SFR_UV_err_up[mask_COLA1], Sigma_SFR_UV_err_down[mask_COLA1]],
            fmt='D', ls='', ms=10, mfc='teal', mec='k', zorder=900,
            mew=1.5, label='COLA1 (UV)')

ax.errorbar(z_Arr[mask_COLA1], Sigma_SFR_UV[mask_COLA1] + np.log10(1/0.6),
        #     yerr=[Sigma_SFR_UV_err_up[mask_COLA1], Sigma_SFR_UV_err_down[mask_COLA1]],
            fmt='D', ls='', ms=10, mfc='none', mec='k', zorder=900,
            mew=1.5)

ax.errorbar(z_Arr[mask_Hb_sn], Sigma_SFR[mask_Hb_sn],
            yerr=[Sigma_SFR_err_up[mask_Hb_sn], Sigma_SFR_err_down[mask_Hb_sn]],
            fmt='o', ls='', label=r'C1F (H$\beta$)', c='crimson', zorder=91,
            ms=5)
ax.errorbar(z_Arr[mask_Hb_sn] + 0.02, Sigma_SFR_UV[mask_Hb_sn],
            yerr=[Sigma_SFR_UV_err_up[mask_Hb_sn], Sigma_SFR_UV_err_down[mask_Hb_sn]],
            fmt='o', ls='', label=r'C1F (UV)', c='teal', zorder=90, ms=5)


#
z_xx = np.linspace(0, 8, 1000)

# Sigma_mean = np.nanmean(Sigma_SFR[mask_Hb_sn])
# Sigma_std = np.nanstd(Sigma_SFR[mask_Hb_sn])
# ax.axhline(Sigma_mean, ls='--', c='r', zorder=10, lw=2)
# ax.fill_between(z_xx, np.ones_like(z_xx) * Sigma_mean + Sigma_std,
#                 np.ones_like(z_xx) * Sigma_mean - Sigma_std,
#                 color='r', alpha=0.2, lw=0)

# mask = (Sigma_SFR > -2) & (Sigma_SFR < 3)
# Sigma_mean = np.nanmean(Sigma_SFR[mask])
# Sigma_std = np.nanstd(Sigma_SFR[mask])
# ax.axhline(Sigma_mean, ls='--', c='teal', zorder=10, lw=2)
# ax.fill_between(z_xx, np.ones_like(z_xx) * Sigma_mean + Sigma_std,
#                 np.ones_like(z_xx) * Sigma_mean - Sigma_std,
#                 color='teal', alpha=0.2, lw=0)


#

# Shibuya+15
r_0 = 6.9 * (1 + z_xx) ** -1.2 * 1**0.27
ax.plot(z_xx, np.log10(10 / 2 / (np.pi * r_0**2)),
        lw=3, ls='--', label='Shibuya+15', alpha=0.5,
        c='purple')

ax.plot(z_xx, np.log10(4 / 2 / (np.pi * r_0**2)),
        lw=3, ls='-', label='Shibuya+15 (re-scaled)', alpha=1,
        color='purple')

# ax.plot([], [], ls='', marker='', label=' ')

# Ion3
Sigma_UV_ion3_percs = np.log10(np.nanpercentile(Sigma_UV_ion3, [16, 50, 84]))
ax.errorbar(4, Sigma_UV_ion3_percs[1],
            yerr=np.array([Sigma_UV_ion3_percs[2] - Sigma_UV_ion3_percs[1],
                           Sigma_UV_ion3_percs[1] - Sigma_UV_ion3_percs[0]]).reshape(2, 1),
            fmt='X', ms=10, mfc='teal', mec='k', ecolor='k', label='Ion3 (Vanzella+18)')
# Ion2
Sigma_UV_ion2_percs = np.log10(np.nanpercentile(Sigma_UV_ion2, [16, 50, 84]))
ax.errorbar(3.2, Sigma_UV_ion2_percs[1],
            yerr=np.array([Sigma_UV_ion2_percs[2] - Sigma_UV_ion2_percs[1],
                           Sigma_UV_ion2_percs[1] - Sigma_UV_ion2_percs[0]]).reshape(2, 1),
            fmt='P', ms=10, mfc='teal', mec='k', ecolor='k', label='Ion2 (Vanzella+16)')

# Kerutt+22
ax.errorbar(redshift_k22, Sigma_k22,
            yerr=[Sigma_k22_err_up, Sigma_k22_err_down],
            fmt='^', ls='', mfc='darkkhaki', mec='k', ecolor='darkkhaki', ms=9,
            label='MUSE-HDUV (Kerutt+23)')

# Izotov+16
ax.errorbar(I16_z_list, Sigma_i16,
            fmt='o', mfc='yellowgreen', mec='k', ms=8,
            label='HST/COS (Izotov+16)')

# Vanzella+22
ax.errorbar(2.37, Sigma_V22,
            fmt='p', mfc='violet', mec='k', ms=9.5,
            label='Sunburst (Vanzella+22)')


ax.set_xlim(-0., 7.2)
ax.set_ylim(-1.7, 2.7)

ax.set_xticks(np.arange(0, 8))

ax.legend(fontsize=10, ncol=2, loc='upper left', frameon=False)
# handles, labels = plt.gca().get_legend_handles_labels()
# order = [3, 4, 5, 6, 0, 1, 2]
# ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order],
#           fontsize=10, ncol=2, frameon=False, loc='upper left') 

ax.tick_params(direction='in', which='both')
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')

ax.set_ylabel(r'$\log_{10}(\Sigma_{\rm SFR}$ / $M_\odot$\,yr$^{-1}$\,kpc$^{-2}$)')
ax.set_xlabel('Redshift')

savefig_path = '/home/alberto/cosmos/ista/COLA1/paper/figures'
fig.savefig(f'{savefig_path}/Sigma_SFR.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
plt.show()

In [ ]:
C1F_Sigma_Hb_percs = np.percentile(Sigma_SFR[mask_Hb_sn], [16, 50, 84])
C1F_Sigma_UV_percs = np.percentile(Sigma_SFR_UV[mask_Hb_sn], [16, 50, 84])
print('Sigma_SFR_Hb', C1F_Sigma_Hb_percs[1], -C1F_Sigma_Hb_percs[1] + C1F_Sigma_Hb_percs)
print('Sigma_SFR_UV', C1F_Sigma_UV_percs[1], -C1F_Sigma_UV_percs[1] + C1F_Sigma_UV_percs)

In [ ]:
src = np.where(cat['NUMBER_1_1'] == 9269)[0]

this_SFR_C_UV = SFR_C_UV_COLA1
this_Hb = 4.10216
this_Hb_err = 0.16479
this_Hg = 2.11797
this_Hg_err = 0.20597

this_redshift = 6.59165
this_R_eff = R_eff[mask_COLA1][0]

this_redshift = cat['z_O3doublet_combined_n_1'][src]
N_iter = 5000
this_rand_beta = np.random.normal(cat['beta'][src], cat['beta_err'][src], size=N_iter)
this_rand_norm = np.random.normal(cat['norm'][src], cat['norm_err'][src], size=N_iter)

this_obs_1500 = 1500 * (1 + this_redshift)

args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
this_f_UV = continuum_powerlaw(*args) * 1e-20

this_distmod = cosmo.distmod(this_redshift).value

this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
this_m_UV[~np.isfinite(this_m_UV)] = -99

this_M_UV_Arr = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

this_M_UV = np.nanmedian(this_M_UV_Arr)
this_M_UV_err = (np.nanpercentile(this_M_UV_Arr, 84) - np.nanpercentile(this_M_UV_Arr, 16)) * 0.5


SFR_Hb_percs, SFR_UV_percs, _ = SFR_XION_dust_corrected(this_Hg, this_Hg_err,
                                                        this_Hb, this_Hb_err,
                                                        this_M_UV, this_M_UV_err,
                                                        this_redshift, this_SFR_C_Hb,
                                                        this_SFR_C_UV,
                                                        N_iter=100000)

N_iter = 10000
this_a = np.random.normal(1.6, 0.3, size=N_iter)
this_b = np.random.normal(0.4, 0.1, size=N_iter)

this_SFR_Hb = np.random.normal(SFR_Hb_percs[1], (SFR_Hb_percs[2] - SFR_Hb_percs[0]) * 0.5,
                                 size=N_iter)
this_Sigma_Hb = np.log10(this_SFR_Hb / (2 * np.pi * this_R_eff**2))
this_SFR_UV = np.random.normal(SFR_UV_percs[1], (SFR_UV_percs[2] - SFR_UV_percs[0]) * 0.5,
                                 size=N_iter)
this_Sigma_UV = np.log10(this_SFR_UV / (2 * np.pi * this_R_eff**2))

this_Sigma = np.random.normal(np.mean(np.concatenate([this_Sigma_Hb, this_Sigma_UV])),
                              np.std(np.concatenate([this_Sigma_Hb, this_Sigma_UV])),
                              N_iter)

f_esc_Sigma = this_a * (10**this_Sigma / 220)**this_b
print(np.median(f_esc_Sigma), -np.median(f_esc_Sigma) + np.percentile(f_esc_Sigma, [16, 84]))

In [ ]:
mask_COLA1_od = (cat['z_O3doublet_1'] >= 6.57) & (cat['z_O3doublet_1'] <= 6.63)

# Naidu
N_iter = 10000
a_rand = np.random.normal(1.6, 0.3, N_iter)
b_rand = np.random.normal(0.4, 0.1, N_iter)
Sigma_SFR_max = 220

cat_beta = fits.open('../uv_slope/UV_slopes_COLA1_field_2fil.fits')[1].data

for src in np.where(mask_COLA1_od)[0]:
    src_beta = np.where(cat_beta['NUMBER'] == cat['NUMBER_1_1'][src])[0][0]

    this_SFR_C_Hb = SFR_C_Hb
    this_SFR_C_UV = SFR_C_UV
    this_Hb = cat['f_Hb_1'][src]
    this_Hb_err = cat['f_Hb_err_1'][src]
    this_Hg = 0.4 * this_Hb
    this_Hg_err = 0
    this_beta = cat_beta['beta'][src_beta]
    this_beta_err = cat_beta['beta_err'][src_beta]
    this_norm = cat_beta['norm'][src_beta]
    this_norm_err = cat_beta['norm_err'][src_beta]
    this_redshift = cat['z_O3doublet_1'][src]

    this_rand_beta = np.random.normal(this_beta, this_beta_err, size=N_iter)
    this_rand_norm = np.random.normal(this_norm, this_norm_err, size=N_iter)

    this_obs_1500 = 1500 * (1 + this_redshift)

    args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
    this_f_UV = continuum_powerlaw(*args) * 1e-20
    
    this_distmod = cosmo.distmod(this_redshift).value

    this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
    this_m_UV[~np.isfinite(this_m_UV)] = -99

    this_M_UV_Arr = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

    this_M_UV = np.nanmedian(this_M_UV_Arr)
    this_M_UV_err = (np.nanpercentile(this_M_UV_Arr, 84) - np.nanpercentile(this_M_UV_Arr, 16)) * 0.5

    SFR_Hb_percs, SFR_UV_percs, _ = SFR_XION_dust_corrected(this_Hg, this_Hg_err,
                                                            this_Hb, this_Hb_err,
                                                            this_M_UV, this_M_UV_err,
                                                            this_redshift, this_SFR_C_Hb,
                                                            this_SFR_C_UV,
                                                            N_iter=100000)

    this_SFR_UV = np.random.normal(SFR_UV_percs[1], (SFR_UV_percs[2] - SFR_UV_percs[0]) * 0.5,
                                 size=N_iter)

    this_Sigma_UV = np.log10(this_SFR_UV / (2 * np.pi * R_eff[src]**2))

    f_esc_Sigma_UV = np.nanpercentile(a_rand * (this_Sigma_UV / Sigma_SFR_max)**b_rand, [16, 50, 84])
    print('NUMBER', cat['NUMBER_1_1'][src])
    print(f_esc_Sigma_UV[1], -f_esc_Sigma_UV[1] + f_esc_Sigma_UV)
    print()

In [ ]:
# Chisholm+22
N_iter = 1000000
for src in np.where(mask_COLA1_od)[0]:
    src_beta = np.where(cat_beta['NUMBER'] == cat['NUMBER_1_1'][src])[0][0]
    this_beta = cat_beta['beta'][src_beta]
    this_beta_err = cat_beta['beta_err'][src_beta]

    a = np.random.normal(1.3, 0.6, N_iter) * 1e-4
    b = np.random.normal(-1.22, 0.1, N_iter)
    beta = np.random.normal(this_beta, this_beta_err, N_iter)

    f_esc_percs = np.percentile(a * 10. ** (b * beta), [16, 50, 84])

    print('NUMBER', cat['NUMBER_1_1'][src])
    print(f_esc_percs[1], -f_esc_percs[1] + f_esc_percs)
    print()